In [ ]:
# Prepare data, train LinearRegression and LogisticRegression (binary target)
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression, LogisticRegression
# from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# # ensure target numeric
# df_1['Risk Category'] = pd.to_numeric(df_1['Risk Category'], errors='coerce')

# # select numeric feature columns (drop target)
# num_cols = df_1.select_dtypes(include=['number']).columns.tolist()
# if 'Risk Category' in num_cols:
#     num_cols.remove('Risk Category')

# X = df_1[num_cols]
# y = df_1['Risk Category']

# # drop rows with missing values
# data = pd.concat([X, y], axis=1).dropna()
# X = data[num_cols]
# y = data['Risk Category']

# # train/test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # scale features
# scaler = StandardScaler()
# X_train_s = scaler.fit_transform(X_train)
# X_test_s = scaler.transform(X_test)

# # Linear regression (not ideal for binary target, used here for demonstration)
# lin = LinearRegression()
# lin.fit(X_train_s, y_train)
# y_pred_reg = lin.predict(X_test_s)

# print("LinearRegression evaluation (regression metrics):")
# print("R2:", r2_score(y_test, y_pred_reg))
# print("MSE:", mean_squared_error(y_test, y_pred_reg))

# # Logistic regression (appropriate for binary classification)
# log = LogisticRegression(max_iter=2000)
# log.fit(X_train_s, y_train)
# y_pred_clf = log.predict(X_test_s)

# print("\nLogisticRegression evaluation (classification metrics):")
# print("Accuracy:", accuracy_score(y_test, y_pred_clf))
# print("Precision:", precision_score(y_test, y_pred_clf, zero_division=0))
# print("Recall:", recall_score(y_test, y_pred_clf, zero_division=0))
# print("F1:", f1_score(y_test, y_pred_clf, zero_division=0))
# print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred_clf))
# print("\nClassification report:\n", classification_report(y_test, y_pred_clf, zero_division=0))